In [87]:
import os
import glob
import pandas as pd
import numpy as np
import pickle
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.collocations import * #Bad form
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import json
import pandas as pd
from pprint import pprint
import re
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction import FeatureHasher
LOCAL_DATA_PATH = '/Users/willsankey/political_history/data'

#Notes:
Thinking about the prospect of using the most common words in the debates for feature extractions, see here:
    http://www.nltk.org/book/ch06.html
        
I need to turn these debate paragraphs into features and am still thinking through it.

In [2]:
DATA_LIST = [
            "['102317'].json"#,
#             "['102344'].json",
#             "['119038'].json",
#             "['72776'].json",
#             "['84482'].json",
#             "['102322'].json",
#             "['118971'].json",
#             "['63163'].json",
#             "['78691'].json",
#             "['84526'].json",
#             "['102343'].json",
#             "['119012'].json",
#             "['72770'].json",
#             "['84382'].json"
            ]

In [3]:
REPUBLICANS = [
    'PRESIDENT BUSH',
    'MCCAIN',
    'RYAN',
    'PALIN',
    'GOV ROMNEY',
    'PENCE',
    'TRUMP'
    
]

In [4]:
DEMOCRATS = [
    'SENATOR KERRY',
    'THE PRESIDENT',
    'VICE PRESIDENT JOSEPH BIDEN',
    'OBAMA',
    'BIDEN',
    'KAINE',
    'CLINTON'
]

In [5]:
## Getting the full dataframe

In [6]:
#### Initialize lists to collect data
other_speakers = []
data_list = []

#For each debate in the data...
for debate in DATA_LIST:
    
    with open(LOCAL_DATA_PATH + "/" + debate) as data_file:
        #Load the debate
        data = json.load(data_file)
        for _, paragraph in data.iteritems():
            ##CLASSIFYING SPEAKER INTO DEMOCRAT, REPUBLICAN
            paragraph['speaker'] = re.sub('[!@#$.:]', '', paragraph['speaker']).upper()
            if paragraph['speaker'] in REPUBLICANS:
                paragraph['party'] = 'Republican'
            elif paragraph['speaker'] in DEMOCRATS:
                paragraph['party'] = 'Democrat'
            else:
                other_speakers.append(paragraph['speaker'])
                paragraph['party'] = 'NA'
                
            ##TRIGRAM FEATURES
            tokens = RegexpTokenizer(r'\w+')
            token = tokens.tokenize(paragraph['text'])
            stops = set(stopwords.words('english'))
            filtered_words = [word for word in token if word not in stops]
            #print filtered_words

            bifinder = BigramCollocationFinder.from_words(filtered_words, window_size = 2)
            #trifinder = TrigramCollocationFinder.from_words(filtered_words, window_size = 3)

            bifinder.apply_freq_filter(2)
            #for Bigram use this syntax --> bigram_measures = nltk.collocations.BigramAssocMeasures()
            #for Trigram use this syntax --> trigram_measures = nltk.collocations.TrigramAssocMeasures()
            trigram_measures = nltk.collocations.TrigramAssocMeasures()
            bigram_measures = nltk.collocations.BigramAssocMeasures()
            for key, val in bifinder.ngram_fd.items():
                print key, val
            
            ##APPEND DATA TO LIST
            data_list.append(paragraph)

(u'great', u'schools') 2
(u'great', u'teachers') 2
(u'I', u'believe') 2
(u'The', u'role') 2
(u'role', u'Government') 2
(u'Government', u'capacity') 2
(u'opportunity', u'create') 2
(u'Let', u'start') 2
(u'endowed', u'Creator') 2
(u'I', u'like') 2
(u'health', u'care') 2
(u'traditional', u'Medicare') 2
(u'voucher', u'program') 2
(u'Dodd', u'Frank') 2
(u'clear', u'difference') 2
(u'insurance', u'afford') 2
(u'dropping', u'insurance') 2
(u'2', u'500') 2
(u'I', u'going') 3
(u'going', u'stop') 2
(u'PBS', u'I') 2
(u'I', u'think') 3
(u'issue', u'I') 3
(u'raising', u'taxes') 2
(u'income', u'Americans') 3
(u'middle', u'income') 2
(u'income', u'people') 2
(u'tax', u'cut') 2
(u'I', u'want') 2
(u'burden', u'paid') 2
(u'income', u'Americans') 2
(u'adds', u'deficit') 2
(u'cut', u'adds') 2
(u'reduce', u'burden') 2
(u'tax', u'cut') 3
(u'Government', u'land') 2
(u'health', u'care') 3
(u'role', u'Government') 2
(u'He', u'said') 2
(u'broaden', u'base') 2
(u'said', u'going') 2
(u'Ronald', u'Reagan') 2
(u'2'

In [7]:
#Data check. Checking that we've captured all presidents in speeches.
#print 'Other speakers that were not classified:', set(other_speakers)

In [8]:
all_data = pd.DataFrame(data_list)

In [9]:
df = all_data[all_data['party'] != 'NA']
del all_data #Cleanup

In [10]:
df.speaker.unique()

array([u'THE PRESIDENT', u'GOV ROMNEY'], dtype=object)

In [11]:
df.describe()

,date,party,speaker,text,title
count,256,256,256,256,256
unique,1,2,2,252,1
top,"October 3, 2012",Republican,GOV ROMNEY,Absolutely.,"Barack Obama: Presidential Debate in Denver, C..."
freq,256,130,130,4,256


In [12]:
df

,date,party,speaker,text,title
0,"October 3, 2012",Democrat,THE PRESIDENT,"I do, because I think that that is the kind of...","Barack Obama: Presidential Debate in Denver, C..."
2,"October 3, 2012",Republican,GOV ROMNEY,"Well, first, I love great schools. Massachuse...","Barack Obama: Presidential Debate in Denver, C..."
3,"October 3, 2012",Republican,GOV ROMNEY,The role of Government: Look behind us—the Con...,"Barack Obama: Presidential Debate in Denver, C..."
4,"October 3, 2012",Democrat,THE PRESIDENT,But I also believe that Government has the cap...,"Barack Obama: Presidential Debate in Denver, C..."
5,"October 3, 2012",Democrat,THE PRESIDENT,"So in the middle of the Civil War, Abraham Lin...","Barack Obama: Presidential Debate in Denver, C..."
6,"October 3, 2012",Democrat,THE PRESIDENT,And so what I've tried to do as President is t...,"Barack Obama: Presidential Debate in Denver, C..."
7,"October 3, 2012",Democrat,THE PRESIDENT,But what I've also said is let's hire another ...,"Barack Obama: Presidential Debate in Denver, C..."
8,"October 3, 2012",Republican,GOV ROMNEY,"Second, in that line that says, ""We are endowe...","Barack Obama: Presidential Debate in Denver, C..."
9,"October 3, 2012",Republican,GOV ROMNEY,But we also believe in maintaining for individ...,"Barack Obama: Presidential Debate in Denver, C..."
10,"October 3, 2012",Republican,GOV ROMNEY,"I'll elaborate, exactly right. First of all, ...","Barack Obama: Presidential Debate in Denver, C..."


In [13]:
##With the full dataset now I'll begin analysis

In [14]:
min_set = df[['text', 'party']]

In [19]:
min_set.head()
# dict_from_data = min_set.to_dict()
  

,text,party
0,"I do, because I think that that is the kind of...",Democrat
2,"Well, first, I love great schools. Massachuse...",Republican
3,The role of Government: Look behind us—the Con...,Republican
4,But I also believe that Government has the cap...,Democrat
5,"So in the middle of the Civil War, Abraham Lin...",Democrat


In [63]:
#define function to tokenize and stem
def clean_text(text):
    """
    Removes punctuation, converts all characters to lowercase, removes stop words, stems
    
    Args:
        a single string of text 
        
    Returns:
        processed text string
        
    """
    tokens = RegexpTokenizer(r'\w+')
    stops = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    
    token = tokens.tokenize(text)
    filtered_words = [word for word in token if word not in stops]
    stems = [stemmer.stem(t) for t in filtered_words]
    return stems


In [84]:
def bigramReturner (tweetString):
    bigramFeatureVector = []
    for item in nltk.bigrams(tweetString):
        bigramFeatureVector.append(' '.join(item))
    return bigramFeatureVector

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [101]:
STOPS = set(stopwords.words('english'))
min_set['bigrams'] = df['party']
for index, row in min_set.iterrows():
    cleaned = clean_text(row.text)
    min_set.bigrams.replace(bigramReturner(cleaned), inplace=True)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


AttributeError: 'list' object has no attribute 'replace'

In [98]:
min_set.head()

,text,party,bigrams
0,"I do, because I think that that is the kind of...",Democrat,Democrat
2,"Well, first, I love great schools. Massachuse...",Republican,Republican
3,The role of Government: Look behind us—the Con...,Republican,Republican
4,But I also believe that Government has the cap...,Democrat,Democrat
5,"So in the middle of the Civil War, Abraham Lin...",Democrat,Democrat


In [16]:
featuresets = [(dict(feature.encode('ascii', 'ignore')), str(label)) for index, (feature, label) in min_set.iterrows()]
#See http://stackoverflow.com/questions/29337714/how-to-run-naive-bayes-from-nltk-with-python-pandas for reference
#And here for ascii fix: http://stackoverflow.com/questions/2365411/python-convert-unicode-to-ascii-without-errors

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [17]:
# print len(featuresets)
# from random import shuffle
# shuffle(featuresets)
# print featuresets

In [18]:
# from nltk.classify import NaiveBayesClassifier as nbc
# train_set, test_set = featuresets[:3000], featuresets[3000:]
# # Train a classifier
# classifier = nbc.train(train_set)
# # Test classifier on "Neo"
# print classifier.classify(ethnic_features('Silva'))